In [32]:
from scipy.io import arff
import pandas as pd

data_aloi, meta_aloi = arff.loadarff('ALOI/ALOI/ALOI.arff')
data_glass, meta_glass = arff.loadarff('Glass/Glass/glass.arff')
data_ionos, meta_ionos = arff.loadarff('Ionosphere/Ionosphere/ionos.arff')
# Convert to DataFrame
df_aloi = pd.DataFrame(data_aloi)
df_glass = pd.DataFrame(data_glass)
df_ionos = pd.DataFrame(data_ionos)


In [2]:
# TODO expliquer ce que représente les datasets ICI

In [3]:
# préparation du dataset et des label
# suppression des label et nettoyage des labels avec 1 et 0
from sklearn.preprocessing import LabelEncoder

def prepare_dataset(df, col_name):
    y_aloi = df.copy()
    # Supprimer la colonne 'outlier' du DataFrame
    df_aloi = df.drop(columns=col_name)
    label_encoder = LabelEncoder()
    y_aloi[col_name] = label_encoder.fit_transform(y_aloi[col_name])
    return df_aloi, y_aloi

In [4]:
# calculs des outliers avec LOF selon les paramètes
# return un tableau des outliers detectés
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split

def compute_outliers_LOF(df, n_neig, cont): 
    lof_model = LocalOutlierFactor(n_neighbors=n_neig, contamination=cont)
    outlier_scores = lof_model.fit_predict(df)
    # -1 outlier et 1 inlier
    # convertion en 1 outlier et 0 inlier
    outliers_detected = outlier_scores == -1
    return outliers_detected.astype(int)

In [33]:
# application de LOF
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import ParameterGrid
def run_search_best_params(df, y, label_col):    
    # Paramètres à tester
    param_grid = {
        'n_neighbors': [3, 5, 10],
        'contamination': [0.005, 0.01, 0.05, 0.1]
    }
    
    # Meilleurs résultats initiaux
    meilleur_f1 = 0
    meilleurs_params = None
    
    # combinaisons de paramètres
    for params in ParameterGrid(param_grid):
        outliers_detected = compute_outliers_LOF(df, params['n_neighbors'], params['contamination'])
        
        f1 = f1_score(y[label_col], outliers_detected)
        accuracy = accuracy_score(y[label_col], outliers_detected)
        recall = recall_score(y[label_col], outliers_detected)

        print(f"Paramètres : {params}")
        print(f"Précision : {accuracy * 100:.2f}%")
        print(f"Recall : {recall * 100:.2f}%")
        print(f"F1 score : {f1:.2f}")
        print("----------------------")
        
        # Mettre à jour les meilleurs paramètres si nécessaire
        if f1 > meilleur_f1:
            meilleur_f1 = f1
            meilleurs_params = params
    
    print("Meilleurs paramètres:")
    print(meilleurs_params)
    print("Meilleur F1 score : {:.2f}".format(meilleur_f1))

In [ ]:
# test pour dataset ALOI - ALOI 49534 objects, 1508 outliers (3.04%) ALOI
print("computing ALOI dataset...")
df_aloi, y_aloi = prepare_dataset(df_aloi, 'outlier')
run_search_best_params(df_aloi, y_aloi, 'outlier')

# test pour dataset glass
# 7 attributes, 214 objects, 9 outliers (4.21%)
print("computing GLASS dataset...")
df_glass, y_glass = prepare_dataset(df_glass, 'outlier')
run_search_best_params(df_glass, y_glass, 'outlier')


# test pour dataset Ionosphere
# 32 attributes, 351 objects, 126 outliers (35.90%)
print("computing IONOS dataset...")
df_ionos, y_ionos = prepare_dataset(df_ionos, 'outlier')
run_search_best_params(df_ionos, y_ionos, 'outlier')


computing ALOI dataset...
